# 1-Baseline model from ACL'22

In [ ]:
import os
import csv
from typing import List, Dict

runs_as_inference_server = os.environ.get('TIRA_INFERENCE_SERVER', None) is not None
dataset_dir = os.environ.get('TIRA_INPUT_DIRECTORY', './dataset')
output_dir = os.environ.get('TIRA_OUTPUT_DIRECTORY', './output')

## Setup

In [ ]:
values = [ "Self-direction: thought", "Self-direction: action", "Stimulation", "Hedonism", "Achievement", "Power: dominance", "Power: resources", "Face", "Security: personal", "Security: societal", "Tradition", "Conformity: rules", "Conformity: interpersonal", "Humility", "Benevolence: caring", "Benevolence: dependability", "Universalism: concern", "Universalism: nature", "Universalism: tolerance", "Universalism: objectivity" ]
value_dict = {value: 1 for value in values}

In [ ]:
# generic code for reading and writing

def readInstances(directory):
    instances = []
    for instancesBaseName in os.listdir(directory):
        if instancesBaseName.startswith("arguments") and instancesBaseName.endswith(".tsv"):
            instancesFileName = os.path.join(directory, instancesBaseName)
            with open(instancesFileName, "r", newline='') as instancesFile:
                print("Reading " + instancesFileName)
                reader = csv.DictReader(instancesFile, delimiter = "\t")
                for fieldName in ["Argument ID", "Conclusion", "Stance", "Premise"]:
                    if fieldName not in reader.fieldnames:
                        print("Skipping file " + instancesFileName + " due to missing field '" + fieldName + "'")
                        continue
                for row in reader:
                    instances.append(row)
    return instances

## Predict function

In [ ]:
def predict(input_list: List) -> List[Dict]:
    return [value_dict for _ in input_list]

In [ ]:
# "instance" is a dict with keys "Argument ID", "Conclusion", "Stance", and "Premise"
def labelInstances(instances: List[Dict]):
    print("Labeling " + str(len(instances)) + " instances")
    predictions = [{"Argument ID": instance["Argument ID"], **labels} for instance, labels in zip(instances, predict(instances))]
    return predictions

## Classification on TIRA

In [ ]:
def writeRun(labels: List[Dict[str, int]], outputDataset):
    if not os.path.exists(outputDataset):
        os.makedirs(outputDataset)

    fieldNames = [ "Argument ID" ] + values

    print("Writing run file")
    with open(os.path.join(outputDataset, "run.tsv"), "w") as runFile:
        writer = csv.DictWriter(runFile, fieldnames = fieldNames, delimiter = "\t")
        writer.writeheader()
        for row in labels:
            writer.writerow(row)

In [ ]:
if not runs_as_inference_server:
    writeRun(labelInstances(readInstances(dataset_dir)), output_dir)